In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras_tuner import RandomSearch
from keras_tuner import Objective

import tensorflow as tf
import tensorflow_ranking as tfr
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

# Load data

In [2]:
train = pd.read_csv('train.csv', index_col='id')
original = pd.read_csv('original.csv')
test = pd.read_csv('test.csv', index_col='id')

# Combine Train with Original

In [3]:
original.prognosis = original.prognosis.str.replace(' ', '_')
train_final = pd.concat([train, original])

# Split Features And Target

In [4]:
X = train_final.drop('prognosis', axis=1)
y = train_final.prognosis

# Target transformation

In [5]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

y = to_categorical(y, num_classes=11)

# Evaluation Metrics

In [6]:
def map3(y_true, y_pred, **kwargs):
    map3_metric = tfr.keras.metrics.MeanAveragePrecisionMetric(topn=3)
    return map3_metric(
        y_true,
        y_pred, **kwargs).numpy()


def map3_from_logloss(y_enc, preds):
    # evaluate against competition training data only
    return map3(y_enc, preds)

def fold_logloss(y_enc, preds):
    # evaluate against competition training data only
    return log_loss(y_enc, preds)

# Define MLP model

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.8, random_state=0)

# model = Sequential()
# model.add(Dense(units=128, activation='relu', input_dim=X.shape[1]))
# model.add(Dense(units=32, activation='relu'))
# model.add(Dense(units=y.shape[1], activation='softmax'))
# model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=[tfr.keras.metrics.MeanAveragePrecisionMetric(topn=3)])
# model.fit(X_train, y_train, epochs=20, batch_size=32, steps_per_epoch=X_train.shape[0]//32+1, validation_data=(X_test, y_test))

In [8]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=512, step=32), activation='relu', input_dim=X.shape[1]))
    for i in range(hp.Int('num_hidden_layers', min_value=0, max_value=3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32), activation=hp.Choice(f'activation_{i}', values=['relu', 'sigmoid', 'tanh'])))
    model.add(Dense(units=y.shape[1], activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling="log")),
                  loss='categorical_crossentropy',
                  metrics=[tfr.keras.metrics.MeanAveragePrecisionMetric(topn=3)])
    return model

tuner = RandomSearch(build_model, 
                     objective=Objective("val_mean_average_precision_metric", direction="max"),
                     max_trials=10,
                     executions_per_trial=3,
                     directory=f'C:/Users/Anes3/Documents/keras_tuner_dir_fold',
                     project_name=f'my_hyperparameter_search_fold'
)

# Define the cross-validation object
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Run the hyperparameter search with cross-validation
for train_index, test_index in cv.split(X, y.argmax(1)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    tuner.search(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 05s]
val_mean_average_precision_metric: 0.5778356591860453

Best val_mean_average_precision_metric So Far: 0.6070601940155029
Total elapsed time: 00h 00m 37s
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# Train the final model on the entire dataset
history = model.fit(X, y, epochs=15, batch_size=32, steps_per_epoch=X.shape[0]//32+1, validation_split=0.2)

Epoch 1/15
30/30 [==============================] - 1s 7ms/step - loss: 2.3839 - mean_average_precision_metric_3: 0.2369 - val_loss: 2.3602 - val_mean_average_precision_metric_3: 0.2682
Epoch 2/15
30/30 [==============================] - 0s 2ms/step - loss: 2.2815 - mean_average_precision_metric_3: 0.3900 - val_loss: 2.2483 - val_mean_average_precision_metric_3: 0.3976
Epoch 3/15
30/30 [==============================] - 0s 2ms/step - loss: 2.1794 - mean_average_precision_metric_3: 0.4331 - val_loss: 2.1586 - val_mean_average_precision_metric_3: 0.4514
Epoch 4/15
30/30 [==============================] - 0s 2ms/step - loss: 2.0746 - mean_average_precision_metric_3: 0.4802 - val_loss: 2.0703 - val_mean_average_precision_metric_3: 0.4913
Epoch 5/15
30/30 [==============================] - 0s 2ms/step - loss: 1.9735 - mean_average_precision_metric_3: 0.5132 - val_loss: 1.9691 - val_mean_average_precision_metric_3: 0.5599
Epoch 6/15
30/30 [==============================] - 0s 2ms/step - loss

In [16]:
y_pred_train = model.predict(X)
sorted_pred_idx = np.argsort(-y_pred_train, axis=1)[:,:3]
original_shape = sorted_pred_idx.shape
top3_pred = encoder.inverse_transform(sorted_pred_idx.reshape(-1,1).ravel())
top3_pred = top3_pred.reshape(original_shape)

30/30 [==============================] - 0s 757us/step


In [17]:
map3(y, y_pred_train)

0.636253

In [28]:
map3(y, y_pred_train)

0.7737226

# Prediction

In [18]:
y_pred = model.predict(test)
sorted_pred_idx = np.argsort(-y_pred, axis=1)[:,:3]
original_shape = sorted_pred_idx.shape
top3_pred = encoder.inverse_transform(sorted_pred_idx.reshape(-1,1).ravel())
top3_pred = top3_pred.reshape(original_shape)

10/10 [==============================] - 0s 665us/step


# Submission

In [19]:
submission = pd.read_csv('sample_submission.csv')
submission['prognosis'] = np.apply_along_axis(lambda x: np.array(' '.join(x), dtype="object"), 1, top3_pred)
submission.to_csv('submission_mlp_model_2.csv', columns=['id', 'prognosis'], index=False)